In [1]:
!pip install git+https://github.com/stanfordnlp/pyreft.git


  Cloning https://github.com/stanfordnlp/pyreft.git to /tmp/pip-req-build-q2b9ypwn
  Running command git clone --filter=blob:none --quiet https://github.com/stanfordnlp/pyreft.git /tmp/pip-req-build-q2b9ypwn
  Resolved https://github.com/stanfordnlp/pyreft.git to commit 28e6e0a7cc0bd0792bf1102ae149f6144433ae48
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of gcsfs to determine which version is compatible with other requirements. This could take a while.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.3 MB/s eta 0:00:00
   ━━━

In [2]:
!pip install nnsight

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 6.6 MB/s eta 0:00:00


In [3]:
import torch, transformers, pyreft


In [4]:
training_examples = [
    ["What are common symptoms of a cold?", "🤧😷🤒"],
    ["How do you treat a headache?", "💊🧴💆‍♂️"],
    ["What are the side effects of aspirin?", "💊🤕😴"],
    ["How do you measure blood pressure?", "🩺💉📈"],
    ["What should I do if I have a fever?", "🌡️💊🛏️"],
    ["What are the signs of a heart attack?", "❤️🚨💔"],
    ["How often should I get a health check-up?", "🗓️🩺🧑‍⚕️"],
    ["What is the normal range for blood sugar?", "🍬🔢📏"],
    ["What is a healthy diet?", "🥗🍎🍗"],
    ["How much water should I drink daily?", "💧🥤🕒"],
    ["What are the symptoms of diabetes?", "🍬📉😓"],
    ["How can I improve my sleep quality?", "🛏️💤🌙"],
    ["What exercises are good for the heart?", "🏋️‍♂️💓🏃‍♂️"],
    ["How do I prevent infections?", "🧼🤲🦠"],
    ["What are the benefits of regular exercise?", "💪🏃‍♂️😊"],
    ["How do I manage stress?", "😌🧘‍♂️🛀"],
    ["What are the risks of smoking?", "🚬⚠️🫁"],
    ["How can I boost my immune system?", "🛡️🧄🍊"],
    ["What are the symptoms of high blood pressure?", "🔝💉📈"],
    ["How can I maintain a healthy weight?", "⚖️🍏🏃‍♂️"],
    ["What are the side effects of antibiotics?", "💊🤢😴"],
    ["How do I know if I have an allergy?", "🌸😷🤧"],
    ["What is a balanced diet?", "🍎🥗🍗"],
    ["How can I lower my cholesterol?", "🧈📉🍵"],
    ["What is mental health?", "🧠❤️😊"],
    ["How do I treat a burn?", "🔥🚑🧴"],
    ["What are the causes of insomnia?", "🌙🚫💤"],
    ["How can I quit smoking?", "🚬❌👍"],
    ["What are the symptoms of depression?", "😞🔵😭"],
    ["How do I handle a panic attack?", "😱🧘‍♂️🛏️"],
    ["What are the risks of obesity?", "⚖️⚠️❤️"],
    ["How can I stay fit at home?", "🏠🏋️‍♂️🧘‍♂️"],
    ["What is a healthy BMI?", "⚖️🔢🧍‍♂️"],
    ["How do I deal with anxiety?", "😟🧘‍♂️💤"],
    ["What are the benefits of yoga?", "🧘‍♂️😊💪"],
    ["How do I treat a sprained ankle?", "🦶🧊🛏️"],
    ["What are the symptoms of a stroke?", "🧠🚨😵"],
    ["How can I manage chronic pain?", "💊🔄🧘‍♂️"],
    ["What is the best way to stay hydrated?", "💧🥤🕒"],
    ["How do I strengthen my bones?", "🦴💪🥛"],
    ["What are the effects of dehydration?", "🥵💧🚫"],
    ["How can I improve my posture?", "🧍‍♂️⬆️🧘‍♂️"],
    ["What is the importance of fiber in diet?", "🌾🍎🥗"],
    ["How do I manage arthritis pain?", "🦴🤕💊"],
    ["What are the symptoms of COVID-19?", "🤒🤧😷"],
    ["How can I prevent back pain?", "🪑⬆️🏋️‍♂️"],
    ["What is the importance of vaccines?", "💉🛡️🦠"],
    ["How do I handle a nosebleed?", "👃🩸🧴"],
]

In [11]:
device = "cuda"

prompt_no_input_template = """\n<|user|>:%s\n<|assistant|>:"""

model_name_or_path = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name_or_path, torch_dtype=torch.bfloat16, device_map=device)

# get tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_name_or_path, model_max_length=2048,
    padding_side="right", use_fast=False)
tokenizer.pad_token = tokenizer.unk_token


In [8]:
from google.colab import drive
drive.mount('/content/drive')

with open('/content/drive/My Drive/hf_token.txt', 'w') as f:
    f.write("hf_kxzazYRYbWaKwMPQUXTVoLvRcEuYWmlF")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
from huggingface_hub import login

with open('/content/drive/My Drive/hf_token.txt', 'r') as f:
    token = f.read().strip()

login(token=token)


In [12]:
# get reft model
reft_config = pyreft.ReftConfig(representations={
    "layer": 8, "component": "block_output",
    "low_rank_dimension": 4,
    "intervention": pyreft.LoreftIntervention(embed_dim=model.config.hidden_size,
    low_rank_dimension=4)})
reft_model = pyreft.get_reft_model(model, reft_config)
reft_model.set_device("cuda")
reft_model.print_trainable_parameters()


trainable intervention params: 16,388 || trainable model params: 0
model params: 1,100,048,384 || trainable%: 0.001489752654370519


In [13]:
data_module = pyreft.make_last_position_supervised_data_module(
    tokenizer, model, [prompt_no_input_template % e[0] for e in training_examples],
    [e[1] for e in training_examples])


In [14]:
training_args = transformers.TrainingArguments(
    num_train_epochs=100.0, output_dir="./tmp", per_device_train_batch_size=10,
    learning_rate=4e-3, logging_steps=40, report_to=[])
trainer = pyreft.ReftTrainerForCausalLM(
    model=reft_model, tokenizer=tokenizer, args=training_args, **data_module)
_ = trainer.train()

Step,Training Loss
40,1.337600
80,0.768700
120,0.454300
160,0.253900
200,0.155300
240,0.102500
280,0.066300
320,0.043600
360,0.028500
400,0.021100


Directory './tmp/checkpoint-500/intervenable_model' created successfully.


In [20]:
instruction = "what not to eat in headache"

# tokenize and prepare the input
prompt = prompt_no_input_template % instruction
prompt = tokenizer(prompt, return_tensors="pt").to(device)

base_unit_location = prompt["input_ids"].shape[-1] - 1  # last position
_, reft_response = reft_model.generate(
    prompt, unit_locations={"sources->base": (None, [[[base_unit_location]]])},
    intervene_on_prompt=True, max_new_tokens=512, do_sample=True,
    eos_token_id=tokenizer.eos_token_id, early_stopping=True
)
print(tokenizer.decode(reft_response[0], skip_special_tokens=True))


<|user|>:what not to eat in headache
<|assistant|>:ahlerts-off foods
  * caffeine, alcohol or nicotine
  * processed foods
  * sugary foods
  * high-fat, high-calorie foods
  * excessive salt or vinegar
  * raw or cooked foods
  * spicy or spicy foods
  * high-fat dairy foods
  * night-soil or smelly foods

here are some foods that you should avoid in an occasional headache:

things to avoid in a headache include:
  - avocado (mildly hot): high in nutrients for healthy brain and mood, and associated with nausea. Can lead to stomach pains
  - tomatoes: mildly hot and potentially dilated blood vessels, associated with increased sensitivity to light, sound or smells
  - onions (mildly hot): usually safe to eat after cooking and high in essential vitamins and nutrients, and can cause stomach upset or skin irritation
  - garlic (mildly hot): also an essential vitamin supplement and sometimes safe to eat after cooking, but may cause skin irritation or stomach upset
  - potatoes (high salt a